## Quora Insincere Questions Classification
#### Aleix Casellas Comas, Rubén Barco Terrones, Andreu Masdeu Ninot, Pablo Lázaro Terrones, Marco Gani Remane

**Motivation** 

Valuable information is always lost when applying standard preprocessing steps - *removing stopwords, stemming, etc* - and doing feature extraction (i.e. TFIDF).

Goal here is to get our quora vocabulary to match close as possible to provided embeddings.
We will work with the GloVe embedding.

*Based on* : https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings

In [1]:
import pandas as pd
import numpy as np
import operator 
import re

In [2]:
train = pd.read_csv('quora-insincere-questions-classification/train.csv').drop('target', axis=1)
test = pd.read_csv('quora-insincere-questions-classification/test.csv')
df = pd.concat([train ,test])
print("Number of texts: ", df.shape[0])

Number of texts:  1681928


In [3]:
# Load embedding

def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
    return embeddings_index

In [4]:
glove = 'quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt'

print("Extracting GloVe embedding")
embed_glove = load_embed(glove)

Extracting GloVe embedding


In [5]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [6]:
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [7]:
vocab = build_vocab(df['question_text'])

In [8]:
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)

Glove : 
Found embeddings for 31.54% of vocab
Found embeddings for  88.16% of all text


In [9]:
# top problematic entries
oov_glove[:10]

[('India?', 21063),
 ('it?', 16623),
 ("What's", 16104),
 ('do?', 11211),
 ('life?', 9993),
 ('you?', 8083),
 ('me?', 7982),
 ('them?', 7910),
 ('time?', 7379),
 ('world?', 6941)]

In [10]:
# Fix contractions

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [11]:
def known_contractions(embed):
    known = []
    for contract in contraction_mapping:
        if contract in embed:
            known.append(contract)
    return known

In [12]:
print("- Known Contractions -")
print("   Glove :")
print(known_contractions(embed_glove))

- Known Contractions -
   Glove :
["can't", "'cause", "didn't", "doesn't", "don't", "I'd", "I'll", "I'm", "I've", "it's", "ma'am", "o'clock", "that's", "you'll", "you're"]


In [13]:
# Fix words with punctuation
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [14]:
def unknown_punct(embed, punct):
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown

In [15]:
print("Glove :")
print(unknown_punct(embed_glove, punct))

Glove :
“ ” ’ ∞ θ ÷ α • à − β ∅ ³ π ‘ ₹ ´ ° £ € × ™ √ ² — – 


In [16]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [17]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'…': ' ... ', 
                '\u200b': ' ', 
                '\ufeff': ''}   
    for s in specials:
        text = text.replace(s, specials[s])
        
    return text

In [18]:
df['treated_question_text'] = df['question_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [19]:
# Rebuild vocabulary and check coverage
vocab = build_vocab(df['treated_question_text'])

print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)

Glove : 
Found embeddings for 72.45% of vocab
Found embeddings for  99.58% of all text


Much better coverage...

In [20]:
oov_glove[:10]

[('Quorans', 1061),
 ('Brexit', 639),
 ('cryptocurrencies', 602),
 ('Redmi', 483),
 ('OnePlus', 159),
 ('UCEED', 152),
 ('Blockchain', 143),
 ('Pokémon', 137),
 ('demonetisation', 132),
 ('GDPR', 130)]

----

After guaranteeing that our quora questions are matching reasonably to our embedding file we can now create a model.

#### CNN with GloVe embedding file

*Based on* : https://www.kaggle.com/shujian/different-embeddings-with-attention-fork-fork

In [24]:
train = pd.read_csv('quora-insincere-questions-classification/train.csv')
test = pd.read_csv('quora-insincere-questions-classification/test.csv')

train['question_text'] = train['question_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [22]:
# Tokenize
vocab = build_vocab(train['question_text'])

print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)

Glove : 
Found embeddings for 74.74% of vocab
Found embeddings for  99.58% of all text


In [25]:
#embed_glove
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

## split to train and val
train, val_df = train_test_split(train, test_size=0.2, random_state=1234)

## some config values 
embed_size = 300 # how big is each word vector
max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

## fill up the missing values
train_X = train["question_text"].fillna("_##_").values
val_X = val_df["question_text"].fillna("_##_").values
test_X = test["question_text"].fillna("_##_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train['target'].values
val_y = val_df['target'].values

In [37]:
#shuffling the data
np.random.seed(2018)
trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))

train_X = train_X[trn_idx]
val_X = val_X[val_idx]
train_y = train_y[trn_idx]
val_y = val_y[val_idx]

In [38]:
all_embs = np.stack(embed_glove.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embed_glove.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/Users/marcoremane/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [40]:
# https://www.kaggle.com/yekenot/2dcnn-textclassifier
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D

filter_sizes = [1,2,3,5]
num_filters = 36

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Reshape((maxlen, embed_size, 1))(x)

maxpool_pool = []
for i in range(len(filter_sizes)):
    conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                 kernel_initializer='he_normal', activation='elu')(x)
    maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

z = Concatenate(axis=1)(maxpool_pool)   
z = Flatten()(z)
z = Dropout(0.1)(z)

outp = Dense(1, activation="sigmoid")(z)

model = Model(inputs=inp, outputs=outp)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
## Train the model 
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 1044897 samples, validate on 261225 samples
Epoch 1/2
  48640/1044897 [>.............................] - ETA: 1:22:16 - loss: 0.2013 - acc: 0.9309

KeyboardInterrupt: 

In [43]:
from sklearn import metrics

pred_cnn_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_cnn_val_y>thresh).astype(int))))

261225/261225 [==============================] - 341s 1ms/step
F1 score at threshold 0.1 is 0.5463678516228748
F1 score at threshold 0.11 is 0.5536458333333333
F1 score at threshold 0.12 is 0.5613044225621884
F1 score at threshold 0.13 is 0.5650850038061406
F1 score at threshold 0.14 is 0.5672102824533389
F1 score at threshold 0.15 is 0.5692581218684338
F1 score at threshold 0.16 is 0.5696865559958982
F1 score at threshold 0.17 is 0.5692745376955903
F1 score at threshold 0.18 is 0.5685468590584464
F1 score at threshold 0.19 is 0.5662043251304996
F1 score at threshold 0.2 is 0.5628106987099333
F1 score at threshold 0.21 is 0.5595167668213096
F1 score at threshold 0.22 is 0.5566375892149088
F1 score at threshold 0.23 is 0.5537457177945835
F1 score at threshold 0.24 is 0.5498060358997963
F1 score at threshold 0.25 is 0.5454484672216093
F1 score at threshold 0.26 is 0.539724630290668
F1 score at threshold 0.27 is 0.5346541488333564
F1 score at threshold 0.28 is 0.5300824416768989
F1 score 

In [44]:
pred_cnn_test_y = model.predict([test_X], batch_size=1024, verbose=1)

  5120/375806 [..............................] - ETA: 7:39

KeyboardInterrupt: 

In [46]:
model.save('embeddings-cnn.h5')